# PROTOTYPE OF FINAL PROJECT

In [20]:
#IMPORTS

import pandas as pd
import datetime
import seaborn as sns
import requests
import os

In [21]:
#GETTING CURRENCY RATES

def currency_converter():

    APIkey = "j4YVbY4DsPe2bRdqchwWzvccCljEhg2EhlPyfrEN"
    url = f"https://api.freecurrencyapi.com/v1/latest?apikey={APIkey}"
    response = requests.get(url).json()
    df = pd.DataFrame(response['data'], index=[0])
    return df

currency_converter()

,AUD,BGN,BRL,CAD,CHF,CNY,CZK,DKK,EUR,GBP,...,PHP,PLN,RON,RUB,SEK,SGD,THB,TRY,USD,ZAR
0,1.491238,1.864876,5.281559,1.359075,0.94251,6.962087,23.146603,7.085729,0.95271,0.820257,...,56.03624,4.489163,4.685132,60.551375,10.40846,1.356629,35.050503,18.629891,1,17.458546


In [22]:
#GETTING FEAR GRID VALUES

def fgindex() -> pd.DataFrame:
    """returns a dataframe of fear and greed index with columns:
    [value],[value_classification],[timestamp],[time_until_update] """

    url = 'https://api.alternative.me/fng/'
    params = {'limit':100000,'date_format':'world'}
    response = requests.get(url,params).json()
    fg = pd.DataFrame(response['data'])
    return fg

fgindex()

,value,value_classification,timestamp,time_until_update
0,25,Extreme Fear,06-12-2022,-1670245835
1,26,Fear,05-12-2022,NaN
2,26,Fear,04-12-2022,NaN
3,27,Fear,03-12-2022,NaN
4,27,Fear,02-12-2022,NaN
...,...,...,...,...
1762,11,Extreme Fear,05-02-2018,NaN
1763,24,Extreme Fear,04-02-2018,NaN
1764,40,Fear,03-02-2018,NaN
1765,15,Extreme Fear,02-02-2018,NaN


In [44]:
! pip install Historic-Crypto
from Historic_Crypto import HistoricalData

In [47]:
#GETTING CLOSING PRICES OF CRYPTO CURRENCY

crypto = 'BTC'

def prices(crypto)-> pd.DataFrame:
    """returns a dataframe with columns:
    ['time_period_start'], ['time_period_end'], ['time_open'], ['time_close'],
    ['rate_open'], ['rate_high'], ['rate_low'], ['rate_close'] """
    today = datetime.datetime.today().strftime('%Y-%m-%d')
    crypto_df = HistoricalData(f'{crypto}-USD',86400,'2018-01-31-00-00',f"{today}-00-00",verbose = False).retrieve_data()
    return crypto_df

prices(crypto)

Provisional Start: 2018-01-31T00:00:00
Provisional End: 2018-11-27T00:00:00


/Users/marksarifidis/.pyenv/versions/gossips_cryptos/lib/python3.10/site-packages/Historic_Crypto/HistoricalData.py:176: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dataset)


Provisional Start: 2018-11-27T00:00:00
Provisional End: 2019-09-23T00:00:00
Provisional Start: 2019-09-23T00:00:00
Provisional End: 2020-07-19T00:00:00
Provisional Start: 2020-07-19T00:00:00
Provisional End: 2021-05-15T00:00:00


/Users/marksarifidis/.pyenv/versions/gossips_cryptos/lib/python3.10/site-packages/Historic_Crypto/HistoricalData.py:176: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dataset)


Provisional Start: 2021-05-15T00:00:00
Provisional End: 2022-03-11T00:00:00


/Users/marksarifidis/.pyenv/versions/gossips_cryptos/lib/python3.10/site-packages/Historic_Crypto/HistoricalData.py:176: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dataset)


Provisional Start: 2022-03-11T00:00:00
Provisional End: 2023-01-05T00:00:00


/Users/marksarifidis/.pyenv/versions/gossips_cryptos/lib/python3.10/site-packages/Historic_Crypto/HistoricalData.py:176: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dataset)


,low,high,open,close,volume
time,,,,,
2018-02-01,8400.00,10166.25,10099.99,9014.23,38171.970034
2018-02-02,7540.00,9090.08,9014.22,8787.52,52039.268077
2018-02-03,8115.48,9499.00,8787.52,9240.00,23700.985090
2018-02-04,7859.00,9350.00,9227.80,8167.91,24062.181957
2018-02-05,6425.75,8349.16,8167.90,6905.19,59578.698221
...,...,...,...,...,...
2022-12-02,16791.02,17108.25,16980.07,17094.71,23096.436867
2022-12-03,16863.58,17158.42,17094.25,16888.53,14081.450672
2022-12-04,16882.86,17199.99,16889.17,17108.90,16961.108288


In [ ]:
BTC_USD = prices(crypto) 


In [13]:
#CLEANING THE DATA

def data_cleaning():
    #cleaning the price data
    BTC_USD = prices(crypto)
    BTC_USD= BTC_USD[['timestamp', 'rate_close']]#getting two columns
    
    #cleaning the sentiment data
    #response = Monica's function
    fg= pd.DataFrame(response['data'])#necessary only if the input data is not in the shape of data frame
    fg['timestamp'] = pd.to_datetime(fg['timestamp'],dayfirst=True) # Converting string to timestamp.
    
    #merging the price and sentiment data
    df=fg.merge(BTC_USD,how='left',on='timestamp')
    
    #cleaning the merged dataframe
    df.drop(columns=['time_until_update', 'value_classification'], inplace = True)
    df.dropna(inplace=True)
    df.set_index(['timestamp'], inplace = True)
    
    return df

data_cleaning()

UnboundLocalError: local variable 'BTC_USD' referenced before assignment